In [ ]:
import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import json
import os
import pytz
from datetime import datetime
import tensorflow as tf

MODEL_DIR = "saved_model"
MODEL_PATH = os.path.join(MODEL_DIR, "sign_mlp.h5")
LABELS_PATH = os.path.join(MODEL_DIR, "labels.json")

st.set_page_config(layout="wide", page_title="Sign Language Detector")

def in_allowed_time():
    tz = pytz.timezone("Asia/Kolkata")
    now = datetime.now(tz)
    hour = now.hour
    return 18 <= hour < 22, now.strftime("%Y-%m-%d %H:%M:%S %Z")

@st.cache_resource
def load_model_and_labels():
    if not os.path.exists(MODEL_PATH) or not os.path.exists(LABELS_PATH):
        return None, None
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(LABELS_PATH, "r") as f:
        labels = json.load(f)
    return model, labels

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def extract_landmarks_from_image(image_bgr):
    image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1,
                        min_detection_confidence=0.5) as hands:
        res = hands.process(image)
        if res.multi_hand_landmarks:
            lm = res.multi_hand_landmarks[0]
            vec = np.array([[p.x, p.y, p.z] for p in lm.landmark]).flatten()
            return vec, lm
    return None, None

def predict_from_vector(model, labels, vec):
    proba = model.predict(vec.reshape(1, -1))[0]
    idx = int(np.argmax(proba))
    return labels[idx], float(proba[idx]), proba

model, labels = load_model_and_labels()

st.title("Sign Language Detection — Upload Image & Real-time (Operational 18:00–22:00 IST)")
allowed, ts = in_allowed_time()
st.write(f"Current time (Asia/Kolkata): {ts}")
if not allowed:
    st.error("System is configured to run predictions only between 18:00 and 22:00 (Asia/Kolkata). Please come back during that window.")
st.sidebar.header("Model")
if model is None or labels is None:
    st.sidebar.warning("Model not found. Run capture_dataset.py to gather data, then train_model.py to create the model.")
else:
    st.sidebar.success(f"Loaded model with {len(labels)} classes")

tabs = st.tabs(["Upload Image", "Realtime Webcam"])
with tabs[0]:
    st.header("Upload an image (png/jpg)")
    uploaded = st.file_uploader("Upload hand image", type=["png", "jpg", "jpeg"])
    if uploaded is not None:
        file_bytes = np.asarray(bytearray(uploaded.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        vec, lm = extract_landmarks_from_image(img)
        display_img = img.copy()
        if lm is not None:
            mp_drawing.draw_landmarks(display_img, lm, mp_hands.HAND_CONNECTIONS)
            st.image(cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB), caption="Detected hand")
            if model is not None and allowed:
                label, conf, proba = predict_from_vector(model, labels, vec)
                st.success(f"Prediction: **{label}** ({conf*100:.1f}% confidence)")
            elif not allowed:
                st.warning("Prediction disabled — outside operational hours.")
            else:
                st.warning("Model not available.")
        else:
            st.warning("No hand detected in image. Try a clearer image or different pose.")

with tabs[1]:
    st.header("Realtime webcam detection")
    st.write("Click 'Start' to open your webcam. Predictions will be shown on each frame.")
    start = st.button("Start Webcam")
    stop = st.button("Stop Webcam")
    run_live = False
    if start:
        run_live = True
    if stop:
        run_live = False
    placeholder = st.empty()
    if run_live:
        cap = cv2.VideoCapture(0)
        with mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                            min_detection_confidence=0.6, min_tracking_confidence=0.6) as hands:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.flip(frame, 1)
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                res = hands.process(rgb)
                label_text = ""
                if res.multi_hand_landmarks:
                    lm = res.multi_hand_landmarks[0]
                    mp_drawing.draw_landmarks(frame, lm, mp_hands.HAND_CONNECTIONS)
                    vec = np.array([[p.x, p.y, p.z] for p in lm.landmark]).flatten()
                    if model is not None and allowed:
                        label, conf, _ = predict_from_vector(model, labels, vec)
                        label_text = f"{label} ({conf*100:.1f}%)"
                    elif not allowed:
                        label_text = "Outside allowed hours"
                cv2.putText(frame, label_text, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                placeholder.image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), channels="RGB")
                if st.session_state.get("stop_signal", False):
                    break
        cap.release()
    else:
        st.info("Press Start Webcam to begin.")